In [6]:
import import_ipynb #Allows you to import ipynb files (TEMPORARY?)
from queue import PriorityQueue
import AStar as AStar
import ReedsShepp as RS
import HybridAStar as HAS

importing Jupyter notebook from AStar.ipynb
importing Jupyter notebook from ReedsShepp.ipynb
importing Jupyter notebook from HybridAStar.ipynb


In [5]:
#Testing AStar

maze = [[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]]
graph = [[0, 1, 0, 0, 0, 0],
         [1, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 1],
         [0, 0, 0, 0, 1, 0],
         [0, 1, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0]]
start = (0, 0)
end = (5, 5)
pf = AStar.AStar(graph, start, end)
path = pf.run()
print(path)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]


In [7]:
#Testing ReedsShepp
maze = [[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]]

In [ ]:
#Testing Hybrid AStar